In [21]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext, SQLContext, Row, SparkConf
from datetime import datetime

# Start Spark Context and sqlContext

In [8]:
conf = SparkConf().setAppName("nasa_data_analysis").setMaster("local")
sc = SparkContext(conf=conf)

In [9]:
sqlContext = SQLContext(sc)

# Load Data 

In [10]:
rdd_data = sc.textFile("../data/access_log_Aug95")

# Take first element to see the lines structure 

In [12]:
sample = rdd_data.take(1)[0]
sample

'in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839'

# Define the function to map the line to an dataframe 

In [45]:
def split_line(line):
    host = line.split(" ")[0]
    timestamp = line.split("[")[1].split("]")[0]
    date = datetime.strptime(timestamp,"%d/%b/%Y:%H:%M:%S %z")
    truncated_date = date.replace(hour=0,minute=0,second=0)
    temp = line.split("\"")
    request = temp[1].strip()
    url=""
    if(len(request.split(" ")) > 1):
        url = request.split(" ")[1]
        
    nothing,code,transfered_bytes = temp[-1].split(" ")
    
    return [host, date, truncated_date, request, url, code, transfered_bytes]

#### Reference to convert a string to datetime 

https://stackoverflow.com/questions/466345/converting-string-into-datetime

https://docs.python.org/2/library/datetime.html#strftime-strptime-behavior

In [ ]:
# Convert the RDD to an DataFrame 

In [50]:
header = ['host', 'date', 'truncated_date', 'request', 'url', 'code', 'transfered_bytes']

In [51]:
maped = rdd_data.map(split_line)

In [52]:
df_data = maped.toDF(header)

In [53]:
df_data.show(10)

+--------------------+-------------------+-------------------+--------------------+--------------------+----+----------------+
|                host|               date|     truncated_date|             request|                 url|code|transfered_bytes|
+--------------------+-------------------+-------------------+--------------------+--------------------+----+----------------+
|   in24.inetnebr.com|1995-08-01 01:00:01|1995-08-01 01:00:00|GET /shuttle/miss...|/shuttle/missions...| 200|            1839|
|     uplherc.upl.com|1995-08-01 01:00:07|1995-08-01 01:00:00|      GET / HTTP/1.0|                   /| 304|               0|
|     uplherc.upl.com|1995-08-01 01:00:08|1995-08-01 01:00:00|GET /images/ksclo...|/images/ksclogo-m...| 304|               0|
|     uplherc.upl.com|1995-08-01 01:00:08|1995-08-01 01:00:00|GET /images/MOSAI...|/images/MOSAIC-lo...| 304|               0|
|     uplherc.upl.com|1995-08-01 01:00:08|1995-08-01 01:00:00|GET /images/USA-l...|/images/USA-logos...| 304|  

# Number of unique hosts 

In [54]:
df_data.select("host").distinct().count()

75060

# Number of 404 errors

In [55]:
df_404 = df_data.filter("code = 404")
df_404.count()

10056

# 5 most error URL 

In [56]:
df_404_url = df_404.groupBy("url").agg({"code":"count"})
df_404_url.sort("count(code)",ascending=False).show(5)

+--------------------+-----------+
|                 url|count(code)|
+--------------------+-----------+
|/pub/winvn/readme...|       1337|
|/pub/winvn/releas...|       1185|
|/shuttle/missions...|        683|
|/images/nasa-logo...|        319|
|/shuttle/missions...|        253|
+--------------------+-----------+
only showing top 5 rows



# Total of 404 per day 

In [57]:
df_404_per_day = df_404.groupBy("truncated_date").agg({"code":"count"})
df_404_per_day.sort("truncated_date").show()

+-------------------+-----------+
|     truncated_date|count(code)|
+-------------------+-----------+
|1995-08-01 01:00:00|        243|
|1995-08-03 01:00:00|        304|
|1995-08-04 01:00:00|        346|
|1995-08-05 01:00:00|        236|
|1995-08-06 01:00:00|        373|
|1995-08-07 01:00:00|        537|
|1995-08-08 01:00:00|        391|
|1995-08-09 01:00:00|        279|
|1995-08-10 01:00:00|        315|
|1995-08-11 01:00:00|        263|
|1995-08-12 01:00:00|        196|
|1995-08-13 01:00:00|        216|
|1995-08-14 01:00:00|        287|
|1995-08-15 01:00:00|        327|
|1995-08-16 01:00:00|        259|
|1995-08-17 01:00:00|        271|
|1995-08-18 01:00:00|        256|
|1995-08-19 01:00:00|        209|
|1995-08-20 01:00:00|        312|
|1995-08-21 01:00:00|        305|
+-------------------+-----------+
only showing top 20 rows



# Stop Context 

In [6]:
sc.stop()